In [82]:
# Para no tener que reiniciar el kernel
%load_ext autoreload
%autoreload 2
import sys  # System-specific parameters and functions
sys.path.append("../")  # Adding the parent directory to the system path
from tqdm import tqdm
from src import sp_api as sa
import numpy as np
import pandas as pd
import pymongo
import re

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [76]:
json_generos = sa.buscar_genero_api()

<Response [200]>


In [77]:
lista_generos = json_generos['results'][1:]

In [ ]:
peliculas = {}
for genero in tqdm(lista_generos):
    lista_pelis = []
    for ano in range(1991,2026):
        for pagina in range(120):
            try:
                lista_pelis.append(sa.buscar_peliculas(genero, ano, pagina))
            except:
                print(f"Error en la pagina {pagina} del año {ano} del genero {genero}")
    peliculas[genero] = lista_pelis

In [41]:
todas = sa.obtener_datos(peliculas)

In [43]:
df = pd.DataFrame(todas)

In [13]:
df.rename(columns={'0':'id', '1':'title', '2': 'type', '3': 'genre', '4': 'year'}, inplace=True)


In [78]:
dicc_generos = sa.buscar_genero(df)


In [15]:
df['genre'] = df.apply(lambda x: sa.aplicar_generos(x['id'], dicc_generos), axis=1)

In [16]:
df = df[df['genre'].notna()]

In [125]:
# lo primero que tenemos que hacer es conectarnos con Mongo
cliente = pymongo.MongoClient()
# nos conectamos con una BBDD 
imdb = cliente['IMDB']
api = imdb['Api']

In [130]:

for row in tqdm(range(df.shape[0])):
    dicc = {}
    dicc['id'] = df.iloc[row,0]
    dicc['title'] = df.iloc[row,1]
    dicc['type'] = df.iloc[row,2]
    dicc['genre'] = df.iloc[row,3]
    dicc['year'] = int(df.iloc[row,4])
    api.insert_one(dicc)

100%|██████████| 314575/314575 [01:53<00:00, 2769.04it/s]
